In [1]:
# Library 
import numpy as np 
import pandas as pd 

In [6]:
# loading data 
df = pd.read_csv("C:/Users/DELL/OneDrive/Desktop/AI/Placement prediction  Project/placement.csv")

In [7]:
df.head

<bound method NDFrame.head of     Unnamed: 0  cgpa     iq  placement
0            0   6.8  123.0          1
1            1   5.9  106.0          0
2            2   5.3  121.0          0
3            3   7.4  132.0          1
4            4   5.8  142.0          0
..         ...   ...    ...        ...
95          95   4.3  200.0          0
96          96   4.4   42.0          0
97          97   6.7  182.0          1
98          98   6.3  103.0          1
99          99   6.2  113.0          1

[100 rows x 4 columns]>

In [15]:
# delete the un necessary dat named unnamed
df = df.drop("Unnamed: 0", axis=1)

In [16]:
df.head()

,cgpa,iq,placement
0,6.8,123.0,1
1,5.9,106.0,0
2,5.3,121.0,0
3,7.4,132.0,1
4,5.8,142.0,0


In [17]:
# now lets save the updated data
df.to_csv("updated_file.csv", header=False, index=False)

In [18]:
df.shape 

(100, 3)

In [19]:
df.info

<bound method DataFrame.info of     cgpa     iq  placement
0    6.8  123.0          1
1    5.9  106.0          0
2    5.3  121.0          0
3    7.4  132.0          1
4    5.8  142.0          0
..   ...    ...        ...
95   4.3  200.0          0
96   4.4   42.0          0
97   6.7  182.0          1
98   6.3  103.0          1
99   6.2  113.0          1

[100 rows x 3 columns]>

In [20]:
# Data visuliztion using scatterplot over cgpa  and p